<a href="https://colab.research.google.com/github/James-R-Chapman/Kia-Invoices/blob/main/Reconcile_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import openpyxl
from itertools import islice

# Open the workbook
workbook = openpyxl.load_workbook('/content/extracted_data.xlsx')

# Get the 'Invoices' sheet
invoices_sheet = workbook['Invoices']
credits_sheet = workbook['Credits']

# Create a dictionary to store the invoice totals
invoice_totals = {}
has_discrepancies = False  # initialize flag variable

# Iterate through the 'Invoices' sheet, adding up the price values for each invoice
for i, row in enumerate(invoices_sheet.iter_rows(min_row=1, values_only=True), start=1):
    if i == 1:
        continue  # skip header row
    invoice_no = row[0]
    price = row[10]
    if invoice_no not in invoice_totals:
        invoice_totals[invoice_no] = 0
    if price is not None:
        try:
            price = float(price)
        except ValueError:
            print(f"Could not convert price {price} to float in row {i}")
        else:
            invoice_totals[invoice_no] += price

for i, row in enumerate(credits_sheet.iter_rows(min_row=1, values_only=True), start=1):
    if i == 1:
        continue  # skip header row
    invoice_no = row[0]
    price = row[10]
    if invoice_no not in invoice_totals:
        invoice_totals[invoice_no] = 0
    if price is not None:
        try:
            price = float(price)
        except ValueError:
            print(f"Could not convert price {price} to float in row {i}")
        else:
            invoice_totals[invoice_no] += price

# Round the invoice totals to two decimal places
for invoice_no in invoice_totals:
    invoice_totals[invoice_no] = round(invoice_totals[invoice_no], 2)

# Get the 'Kia Invoices' sheet
kia_sheet = workbook['Kia Invoices']

# Iterate through the 'Kia Invoices' sheet, comparing the subtotal value to the invoice total for each invoice
for row in islice(kia_sheet.iter_rows(min_row=1, values_only=True), 1, None):
    invoice_no = row[0]
    subtotal = row[4]
    if invoice_no in invoice_totals:
        if subtotal != invoice_totals[invoice_no]:
            print(f"Invoice {invoice_no} subtotal does not match invoice total: {subtotal} vs {invoice_totals[invoice_no]}")
            has_discrepancies = True  # set flag variable to True
    elif invoice_no not in invoice_totals:
        print(f"No invoice total found for invoice {invoice_no}")
        has_discrepancies = True  # set flag variable to True

if not has_discrepancies:  # check flag variable
    print("All Invoices Reconcile")